 ## モデルの選択方法（BIC・AIC）

## セットアップ

In [ ]:
# 基本的なライブラリ群
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# statsmodels.formula.apiをsmfとしてインポートしましょう．
import statsmodels.formula.api as smf


# scikit-learn.linear_modelのLinearRegressionをインポートする
from sklearn.linear_model import LinearRegression

# dmbaからstepwise_selection, forward_selection, backward_eliminationをインポートする
from dmba import stepwise_selection, forward_selection, backward_elimination

In [ ]:
# Google driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 作業ディレクトリの移動
%cd /content/drive/MyDrive/WASEDADS/04/

# 作業ディレクトリにあるファイル名を表示
%ls

In [ ]:
# データの読み込み
# pandasのメソッドread_excelを使って4-1_WorkData_Wine.xlsxを読み込み，dfという変数に代入しましょう
df = pd.read_excel('4-1_WorkData_Wine.xlsx', index_col=0)
df.head()
# df.info()

## statsmodelsを使ったモデル選択（AIC・BIC）

In [ ]:
from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 多重共線性のチェックを行う関数
def check_vif(df, variable_list, threshold=10):
    # 説明変数データと定数項を準備
    df_X = df[variable_list].copy()
    df_X['Intercept'] = 1

    # VIF計算
    vif_df = pd.DataFrame()
    vif_df['variables_x'] = df_X.columns
    vif_df['VIF_Factor'] = [variance_inflation_factor(df_X.values, i) for i in range(len(df_X.columns))]

    # 出力（切片を除外して表示）
    print("\n▶ 各変数のVIF：")
    print(vif_df[vif_df['variables_x'] != 'Intercept'])

    # VIFがthresholdを超える変数の表示
    high_vif = vif_df[(vif_df['VIF_Factor'] > threshold) & (vif_df['variables_x'] != 'Intercept')]

    print(f"\n▶ VIFが {threshold} を超える変数（多重共線性の可能性あり）：")
    if not high_vif.empty:
        print(high_vif)
    else:
        print(f"特にありません（すべてのVIF ≦ {threshold}）")

    return vif_df

# すべての説明変数の組み合わせを取得する関数
def get_combinations(array):
  all_combinations = []
  for r in range(0, len(variables_x) + 1):  # ←ここを0からに変更
      combos = list(combinations(variables_x, r))
      all_combinations.extend(combos)

  return all_combinations

# 全ての関係式を取得する関数
def get_formulas(array, y_state):
  x_combinations = get_combinations(array)
  lr_formula1s = []
  for combo in x_combinations:
    if len(combo) == 0:
      formula = f"{y_state} ~ 1"  # 切片のみのモデル
    else:
      formula = f"{y_state} ~ {' + '.join(combo)}"
    lr_formula1s.append(formula)
  return lr_formula1s

# 全ての関係式について、AICまたはBICを求める関数
def calc_aic_or_bic(lr_formulas, df, criterion):
  results = []
  for count, lr_formula in enumerate(lr_formulas):
      result = smf.ols(lr_formula, df).fit()

      if criterion == 'AIC':
          score = result.aic
          results.append((lr_formula, score))
      elif criterion == 'BIC':
          score = result.bic
          results.append((lr_formula, score))
      else:
          print("評価基準を確認してください")
  return results


############################ 以下分析の実行コード##################################

print("============= 多重共線性のチェック ================")
variables_x = ['citric_acid', 'residual_sugar', 'chlorides', 'total_sulfur_dioxide', 'alcohol']
vif_df = check_vif(df, variables_x)

print("===============================================\n")


print("目的：構造推定「品質評価がワインの成分からどのように決まるかを調べる」")

criterion = 'BIC' #'AIC', 'BIC'
print("評価基準：",criterion)

variables_x = ['citric_acid', 'residual_sugar', 'chlorides', 'total_sulfur_dioxide', 'alcohol']

lr_formulas = get_formulas(variables_x, 'quality')
print(lr_formulas)
results = calc_aic_or_bic(lr_formulas, df, criterion)
# スコアで昇順にソートして上位N件を表示
N = 5
sorted_results = sorted(results, key=lambda x: x[1])
print(f"\n{criterion} が小さい順の上位 {N} 件：")
for rank, (formula, score) in enumerate(sorted_results[:N], 1):
    print(f"{rank}. {criterion} = {score:.3f}: 「{formula}」")

print(f"\n評価基準を{criterion}とした際に最適な関係式：")
print(sorted_results[0][0])

In [ ]:
print("============= 多重共線性のチェック ================")
variables_x = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year']
vif_df = check_vif(df, variables_x)
print("===============================================\n")
print("'cylinders', 'displacement', 'weight'を除いて再度チェック")
variables_x = ['horsepower', 'acceleration', 'year']
vif_df = check_vif(df, variables_x)
print("===============================================\n")



print("目的：予測「車種の情報から燃費性能を予測する」")

criterion = 'AIC' #'AIC', 'BIC'
print("評価基準：",criterion)

# 多重共線性の強い説明変数を除いた中で最適な組み合わせを見つける
variables_x = ['horsepower', 'acceleration', 'year']

lr_formulas = get_formulas(variables_x, 'mpg')
results = calc_aic_or_bic(lr_formulas, df, criterion)

# スコアで昇順にソートして上位N件を表示
N = 5
sorted_results = sorted(results, key=lambda x: x[1])
print(f"\n{criterion} が小さい順の上位 {N} 件：")
for rank, (formula, score) in enumerate(sorted_results[:N], 1):
    print(f"{rank}. {criterion} = {score:.3f}: 「{formula}」")

print(f"\n評価基準を{criterion}とした際に最適な関係式：")
print(sorted_results[0][0])

## 変数選択法によるモデル選択

In [ ]:
# 1.関係式の推定を行う自作関数
def lr_fit(variables_x):
    # 説明変数がない(0)場合は，実行せずに終了
    if len(variables_x) == 0:
        return None

    # 関係式の推定
    reg_linear = LinearRegression()
    reg_linear.fit(df_X[variables_x], df_Y)
    return reg_linear

# 2.推定を行った関係式から評価基準の値を算出する自作関数
def lr_score(reg_linear, variables_x):
    if len(variables_x) == 0:
      # すべての要素（データ数）がdf_Yの平均値となる配列を作成
      y_pred = [df_Y.mean()] * len(df_Y)
    else:
      # 推定した関係式からデータ（df_X）に対するyの予測値を取得
      y_pred = reg_linear.predict(df_X[variables_x])

    # データ数
    n = len(y_pred)
    # 説明変数の数
    k_params = len(variables_x)
    # 残差平方和をデータ数で割った値
    sigma2 = np.sum((df_Y - y_pred) ** 2) / n
    # 最大対数尤度
    logL = -n / 2 * (np.log(2 * np.pi * sigma2) + 1)

    # 評価基準値の算出
    if criterion == 'AIC':
        aic = -2 * logL + 2 * (k_params + 1)
        return aic
    elif criterion == 'BIC':
        bic = -2 * logL + (k_params + 1) * np.log(n)
        return bic
    else:
        print('評価基準を確認してください')

In [ ]:
print('-----変数増減法-----')
best_model, best_variables = stepwise_selection(variables_x, lr_fit, lr_score,verbose=True)
print('Intercept:',best_model.intercept_)
print('Coefficients: ', best_model.coef_)
print('best_variables: ', best_variables)

In [ ]:
print('-----変数増加法-----')
best_model, best_variables = forward_selection(variables_x, lr_fit, lr_score,verbose=True)
print('Intercept:',best_model.intercept_)
print('Coefficients: ', best_model.coef_)
print('best_variables: ', best_variables)

In [ ]:
print('-----変数減少法-----')
best_model, best_variables = backward_elimination(variables_x, lr_fit, lr_score,verbose=True)
print('Intercept:',best_model.intercept_)
print('Coefficients: ', best_model.coef_)
print('best_variables: ', best_variables)